Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.7014973144332155, θ=0.8083320965354355)   …  Distributions.Gamma(α=0.31533751166972057, θ=0.06491620062297443)
 Distributions.Gamma(α=0.5849309526397368, θ=0.6028699420612047)      Distributions.Gamma(α=0.7741974319701441, θ=0.3506421384753411)  
 Distributions.Gamma(α=0.3193475119815765, θ=0.6507414302336967)      Distributions.Gamma(α=0.02617886480985221, θ=0.954945375984442)  
 Distributions.Gamma(α=0.7797911051866175, θ=0.8309353873833305)      Distributions.Gamma(α=0.7899430389757256, θ=0.2089624685646514)  
 Distributions.Gamma(α=0.29939003412179255, θ=0.7879635643621248)     Distributions.Gamma(α=0.12567006268742897, θ=0.04760406872741485))

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.874865  0.464935  0.434869  0.245545
 0.248193  0.318106  0.176542  0.706653
 0.720009  0.189135  0.741034  0.616015)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.251505   0.851336  0.134329   0.894233
 0.221033   0.36679   0.0989635  0.210126
 0.0390644  0.405171  0.751634   0.285444)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.6104151849285111, σ=0.39721623624026314)   …  Distributions.Normal(μ=0.5149989805486088, σ=0.2823113660187977) 
 Distributions.Normal(μ=0.026757027124166166, σ=0.7618618454698423)     Distributions.Normal(μ=0.8678712198491265, σ=0.5725357338544352) 
 Distributions.Normal(μ=0.23410472394305026, σ=0.6178927039257589)      Distributions.Normal(μ=0.7942549749547629, σ=0.7659511696716654) 
 Distributions.Normal(μ=0.6836854399143228, σ=0.1593149301718333)       Distributions.Normal(μ=0.28423924336535, σ=0.6904390694640186)   
 Distributions.Normal(μ=0.382914848591728, σ=0.04331949175832861)       Distributions.Normal(μ=0.10579699471720705, σ=0.7307051837620491))

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.378146    0.552786  0.322886  0.0108842
 0.00890148  0.18302   0.890218  0.96874  
 0.0497519   0.100489  0.236619  0.228564 )

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-1.6363854053200706

In [10]:
g = @factor EntropyFactor x
value(g)

1.4189385332046727

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-1.7746542360483384,-2.2655121234846454,0.007496947218732133)

In [15]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.546135   0.696757   0.896892  0.107747   0.207077   0.488645  
 0.0550184  0.0423218  0.145555  0.417331   0.315212   0.405151  
 0.657353   0.0331194  0.758347  0.2157

In [17]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [19]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [20]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…

In [17]:
naturals(obs, Val{:x}, μ)

LoadError: LoadError: MethodError: `naturals` has no method matching naturals(::VB.LogNormalFactor{2}, ::Type{Val{:x}}, ::VB.RandomNode{Distributions.Normal})
while loading In[17], in expression starting on line 1

In [18]:
naturals(obs, Val{:μ}, μ)

LoadError: LoadError: MethodError: `naturals` has no method matching naturals(::VB.LogNormalFactor{2}, ::Type{Val{:μ}}, ::VB.RandomNode{Distributions.Normal})
while loading In[18], in expression starting on line 1

In [19]:
naturals(obs, Val{:τ}, τ)

LoadError: LoadError: MethodError: `naturals` has no method matching naturals(::VB.LogNormalFactor{2}, ::Type{Val{:τ}}, ::VB.RandomNode{Distributions.Gamma})
while loading In[19], in expression starting on line 1

In [20]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

false

In [21]:
check_conjugate(m.nodes[:τ])

LoadError: LoadError: UndefVarError: check_conjugate not defined
while loading In[21], in expression starting on line 1

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
update!(μ, Val{:conjugate})

LoadError: LoadError: UndefVarError: update! not defined
while loading In[23], in expression starting on line 1

In [24]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)